<a href="https://colab.research.google.com/github/alvlagus/Introduction_to_NLP_GB/blob/Lesson_9/gusev_NLP_HW_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'>Введение в обработку естественного языка</h1>  

<h2 align='center'>Урок 9. Языковое моделирование</h2>  

<h3 align='left'>Практическое задание:</h3>  

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов



###Загружаем библиотеки:

In [36]:
import tensorflow as tf
import pandas as pd
import numpy as np

import os
import time

###Загружаем данные:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path_to_file = '/content/drive/MyDrive/Colab_Notebooks/25_NLP/work_9/data/evgenyi_onegin.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [7]:
text = text + text

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
# text_as_int, text, len(text_as_int), len(text)

## train and target

In [12]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


Сместим предложения для организации тренировочных данных

In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 1024

# Number of RNN units
rnn_units = embedding_dim

In [19]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "rn_ngenerator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  134144    
                                                                 
 gru (GRU)                   multiple                  6297600   
                                                                 
 gru_1 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  134275    
                                                                 
Total params: 12,863,619
Trainable params: 12,863,619
Non-trainable params: 0
_________________________________________________________________


In [22]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[-0.00063004,  0.00221605, -0.00644166, ..., -0.01061852,
         0.00183544, -0.0039487 ],
       [-0.00057896,  0.00594268, -0.00965196, ..., -0.01359057,
         0.00169725, -0.00373325],
       [-0.00038774,  0.00930386, -0.01145609, ..., -0.01336055,
         0.00120511, -0.00222342],
       ...,
       [-0.00013104,  0.01319372, -0.0134996 , ..., -0.01162102,
         0.00032073,  0.00013022],
       [ 0.00056437,  0.01462831, -0.01425424, ..., -0.01056289,
         0.00066104,  0.00203356],
       [ 0.00097939,  0.01570242, -0.01493575, ..., -0.0097827 ,
         0.00094089,  0.00355276]], dtype=float32)>

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '               Читать в глаза своим любезным\n                        Свои творенья? Говорят,\n       '

Next Char Predictions: 
 'k:пЛnжXЦ!W9h-ЛУGHЦЯС7?с)mАВфHkШTк6леsжВЬЮ,ббАЖЦПжfкM FЮ 0НуEqфГЗДвм!3ФУ kэ3kMчФ!Yож?"х?:ж}чucДИ6чЖщЛ'


## Train the model

In [26]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8686833


In [27]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [33]:
!rm -rf ./training_checkpoints

In [34]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

## Execute the training

In [70]:
EPOCHS = 50

In [71]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
88/88 [==============================] - 29s 317ms/step - loss: 0.2296
Epoch 2/50
88/88 [==============================] - 29s 315ms/step - loss: 0.2110
Epoch 3/50
88/88 [==============================] - 28s 311ms/step - loss: 0.1978
Epoch 4/50
88/88 [==============================] - 29s 319ms/step - loss: 0.1802
Epoch 5/50
88/88 [==============================] - 29s 317ms/step - loss: 0.1594
Epoch 6/50
88/88 [==============================] - 29s 314ms/step - loss: 0.1428
Epoch 7/50
88/88 [==============================] - 30s 321ms/step - loss: 0.1541
Epoch 8/50
88/88 [==============================] - 29s 309ms/step - loss: 0.1386
Epoch 9/50
88/88 [==============================] - 28s 311ms/step - loss: 0.1286
Epoch 10/50
88/88 [==============================] - 29s 320ms/step - loss: 0.1201
Epoch 11/50
88/88 [==============================] - 29s 315ms/step - loss: 0.1145
Epoch 12/50
88/88 [==============================] - 29s 314ms/step - loss: 0.1106
Epoch 13/50
8

## Generate text

In [72]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_49'

In [73]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [74]:
model.summary()

Model: "rn_ngenerator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  134144    
                                                                 
 gru (GRU)                   multiple                  6297600   
                                                                 
 gru_1 (GRU)                 multiple                  6297600   
                                                                 
 dense (Dense)               multiple                  134275    
                                                                 
Total params: 12,863,619
Trainable params: 12,863,619
Non-trainable params: 0
_________________________________________________________________


In [81]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.001

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [82]:
text_ = generate_text(model, start_string=u"И вот идет уже")
print(text_)

И вот идет ужелит                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 


In [83]:
len(text_)

514